In [21]:
import sys
print(sys.version)
import pyarrow
print("Spark Version:", sc.version)
print("PyArrow version:", pyarrow.__version__)
assert(pyarrow.__version__.startswith("0.14"))
import sklearn
print("sklearn version:", sklearn.__version__)


3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 13:42:17) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Spark Version: 2.4.4
PyArrow version: 0.14.1
sklearn version: 0.21.3


In [22]:
# Only once - create parquet
if False:
    flights_path = "file:///Users/talfranji/data/schoolmaster/flights/"
    flights = spark.read.format("csv"
         ).option("header", "true"
         ).option("inferSchema","true"
         ).load(flights_path)
    #flights.show()
    flights_out = "file:///Users/talfranji/data/schoolmaster/flightspq"
    #flights.write.save(flights_out)
    #flights = flights.drop(flights.columns[-1])


In [23]:
flights_parquet_local = "file:///Users/talfranji/data/schoolmaster/flightspq"
#flights_parquet_local = "s3://franji.school/data/flights/"
flights = spark.read.load(flights_parquet_local)
#flights.printSchema()
flights.createOrReplaceTempView("flights")

In [24]:
#--The dataset contains onlt date as FL_DATE (time==00:00 always) - 
# we add the CRS_DEP_TIME which a float but represents the time in 4 digits. e.g. 23:55 is represented as 2,355.0
# we combine FL_DATE and CRS_DEP_TIME into fl_dtime
flights_raw = spark.sql("""
select to_timestamp(CAST(FL_DATE as BIGINT) +CAST(CRS_DEP_TIME/100.0 as INT)*60*60 + (CAST(CRS_DEP_TIME as INT) % 100) * 60 ) 
                    as fl_dtime,
        CAST(CRS_DEP_TIME/100.0 as INT) as HH,
        month(FL_DATE) as MM,
       dayofmonth(FL_DATE) as DD,
       OP_CARRIER, 
       ORIGIN, 
       DEST, 
       case when DEP_DELAY > 30 then 1 else 0 end as is_late
from flights
""")
flights_raw.createOrReplaceTempView("flights_raw")

# Data after preparation

In [25]:
flights_raw.show(5)

+-------------------+---+---+---+----------+------+----+-------+
|           fl_dtime| HH| MM| DD|OP_CARRIER|ORIGIN|DEST|is_late|
+-------------------+---+---+---+----------+------+----+-------+
|2014-01-01 11:35:00| 11|  1|  1|        AA|   ICT| DFW|      0|
|2014-01-01 22:25:00| 22|  1|  1|        AA|   MIA| TPA|      0|
|2014-01-01 21:05:00| 21|  1|  1|        EV|   DFW| HOU|      0|
|2014-01-01 16:55:00| 16|  1|  1|        EV|   CRW| DFW|      1|
|2014-01-01 13:20:00| 13|  1|  1|        EV|   DFW| CRW|      1|
+-------------------+---+---+---+----------+------+----+-------+
only showing top 5 rows



## Data is unbalanced

In [26]:
spark.sql("""
select count(*), sum(is_late), sum(is_late)/ count(*)
from flights_raw
""").show()

+--------+------------+-------------------------------------------------------------------------+
|count(1)|sum(is_late)|(CAST(sum(CAST(is_late AS BIGINT)) AS DOUBLE) / CAST(count(1) AS DOUBLE))|
+--------+------------+-------------------------------------------------------------------------+
|61556964|     6619311|                                                      0.10753147279973067|
+--------+------------+-------------------------------------------------------------------------+



## Taking unbalanced sample

In [27]:
sample_class0 = flights_raw.filter("is_late = 0").sample(True, 0.0005)
sample_class1 = flights_raw.filter("is_late = 1").sample(True, 0.0005)
sample = sample_class0.union(sample_class1)
flights_pd = sample.toPandas()
flights_pd.head()


,fl_dtime,HH,MM,DD,OP_CARRIER,ORIGIN,DEST,is_late
0,2014-01-01 08:45:00,8,1,1,WN,DEN,TUS,0
1,2014-01-01 20:00:00,20,1,1,WN,TUS,LAS,0
2,2014-01-01 06:05:00,6,1,1,OO,EUG,SLC,0
3,2014-01-01 11:00:00,11,1,1,AA,DCA,DFW,0
4,2014-01-01 08:20:00,8,1,1,AA,STL,ORD,0


In [43]:
import datetime
import math
import pandas as pd
from sklearn.base import BaseEstimator

def date_periods(dt):
    """Given a datetime - return 6 coordinates
    - sin/cos of the position of the date/time inside a year/week/day period"""
    year_begin = datetime.datetime(dt.year, 1, 1)
    y_period = (dt - year_begin) / datetime.timedelta(days=365)
    dt_week_start = dt - datetime.timedelta(days=dt.weekday())
    week_begin = datetime.datetime(dt_week_start.year, dt_week_start.month, dt_week_start.day)
    w_period = (dt - week_begin) / datetime.timedelta(days=7)
    day_begin = datetime.datetime(dt.year, dt.month, dt.day)
    d_period = (dt - day_begin) / datetime.timedelta(days=1)
    PI2 = math.pi * 2.0
    return math.sin(y_period * PI2), math.cos(y_period * PI2), math.sin(w_period * PI2), math.cos(w_period * PI2), math.sin(d_period * PI2), math.cos(d_period * PI2)

class DatePeriodTransformer(BaseEstimator):
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        # X is a DataFrame with a single column.
        # We take the Series of the first column and use map() to convert it into an array of tuples
        # then we turn it back into a DataFrame which is what we are expected to return
        period_cols = ['p_y_sin', 'p_y_cos', 'p_w_sin', 'p_w_cos', 'p_d_sin', 'p_d_cos']
        return pd.DataFrame(X[X.columns[0]].map(date_periods).tolist(), columns=period_cols)
    
#print(date_periods(datetime.datetime(2019, 12, 12, 14, 15, 0)))

In [29]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import scipy.sparse
import flights_transformer

column_trans = ColumnTransformer(
     [
     ('origin_onehot', OneHotEncoder(dtype='int', handle_unknown='ignore'),['ORIGIN']),
     #('origin_onehot', 'drop',['ORIGIN']),
     ('op_carrier_onehot', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['OP_CARRIER']),
     #('op_carrier_onehot', 'drop', ['OP_CARRIER']),
     ('dt_period', DatePeriodTransformer(), ['fl_dtime']),
     #('dt_period', 'drop', ['fl_dtime']),
     ('hh_onehot', OneHotEncoder(dtype='int', categories='auto',handle_unknown='ignore'), ['HH']),
     ('dd_onehot', OneHotEncoder(dtype='int', categories='auto',handle_unknown='ignore'), ['DD']),
     ('mm_onehot', OneHotEncoder(dtype='int', categories='auto',handle_unknown='ignore'), ['MM']),
     ],
     remainder='drop', n_jobs=1)

model = Pipeline(steps=[
    ("fextract", column_trans),
    ("tree", RandomForestClassifier(n_estimators=100)),
    #("logreg", LogisticRegression(solver='lbfgs'))
])

X = flights_pd.drop(columns="is_late")
Y = flights_pd['is_late'].values
model.fit(X, Y)

Pipeline(memory=None,
         steps=[('fextract',
                 ColumnTransformer(n_jobs=1, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('origin_onehot',
                                                  OneHotEncoder(categorical_features=None,
                                                                categories=None,
                                                                drop=None,
                                                                dtype='int',
                                                                handle_unknown='ignore',
                                                                n_values=None,
                                                                sparse=True),
                                                  ['ORIGIN']),
                                                 ('op_carrier_onehot

In [30]:
# score on train-set
model.score(X, Y)

1.0

# Score on full data set - take 1

In [32]:
#### Scoring using pandas_udf
#### Take 1
import pyspark.sql.functions as funcs

all_cols = [flights_raw[c] for c in flights_raw.columns]
model_bc = sc.broadcast(model)

@funcs.pandas_udf('double')
def estimate(*cols):
    # build back X from all the feature columns
    X = pd.concat(cols[:-1], axis=1) 
    Y = cols[-1]
    model = model_bc.value
    score = model.score(X, Y)
    # Note - we calculate a single number - but need to return
    # a Series with same size as input
    return pd.Series(score, index=range(X.shape[0]))

flights_raw.filter("is_late = 1").sample(True, 0.1).withColumn('score', 
                        estimate(*all_cols)).agg(funcs.avg("score")).show()
flights_raw.filter("is_late = 0").sample(True, 0.1).withColumn('score',
                        estimate(*all_cols)).agg(funcs.avg("score")).show()

Py4JJavaError: An error occurred while calling o229.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 16.0 failed 1 times, most recent failure: Lost task 3.0 in stage 16.0 (TID 129, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/talfranji/Downloads/spark-2.4.4-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/Users/talfranji/Downloads/spark-2.4.4-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/talfranji/Downloads/spark-2.4.4-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 286, in dump_stream
    for series in iterator:
  File "<string>", line 1, in <lambda>
  File "/Users/talfranji/Downloads/spark-2.4.4-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 101, in <lambda>
    retur

# Score on full data set - take 2
Put custom class in a module

In [45]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import scipy.sparse
import date_transformer

column_trans = ColumnTransformer(
     [
     ('origin_onehot', OneHotEncoder(dtype='int', handle_unknown='ignore'),['ORIGIN']),
     #('origin_onehot', 'drop',['ORIGIN']),
     ('op_carrier_onehot', OneHotEncoder(dtype='int', handle_unknown='ignore'), ['OP_CARRIER']),
     #('op_carrier_onehot', 'drop', ['OP_CARRIER']),
     ('dt_period', date_transformer.DatePeriodTransformer(), ['fl_dtime']),
     #('dt_period', 'drop', ['fl_dtime']),
     ('hh_onehot', OneHotEncoder(dtype='int', categories='auto',handle_unknown='ignore'), ['HH']),
     ('dd_onehot', OneHotEncoder(dtype='int', categories='auto',handle_unknown='ignore'), ['DD']),
     ('mm_onehot', OneHotEncoder(dtype='int', categories='auto',handle_unknown='ignore'), ['MM']),
     ],
     remainder='drop', n_jobs=1)

model = Pipeline(steps=[
    ("fextract", column_trans),
    ("tree", RandomForestClassifier(n_estimators=100)),
    #("logreg", LogisticRegression(solver='lbfgs'))
])

X = flights_pd.drop(columns="is_late")
Y = flights_pd['is_late'].values
model.fit(X, Y)

Pipeline(memory=None,
         steps=[('fextract',
                 ColumnTransformer(n_jobs=1, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('origin_onehot',
                                                  OneHotEncoder(categorical_features=None,
                                                                categories=None,
                                                                drop=None,
                                                                dtype='int',
                                                                handle_unknown='ignore',
                                                                n_values=None,
                                                                sparse=True),
                                                  ['ORIGIN']),
                                                 ('op_carrier_onehot

In [46]:
#### Scoring using pandas_udf
#### Take 2
import pyspark.sql.functions as funcs
import date_transformer

all_cols = [flights_raw[c] for c in flights_raw.columns]
model_bc = sc.broadcast(model)

@funcs.pandas_udf('double')
def estimate(*cols):
    # build back X from all the feature columns
    X = pd.concat(cols[:-1], axis=1) 
    Y = cols[-1]
    model = model_bc.value
    score = model.score(X, Y)
    # Note - we calculate a single number - but need to return
    # a Series with same size as input
    return pd.Series(score, index=range(X.shape[0]))

flights_raw.filter("is_late = 1").sample(True, 0.1).withColumn('score', 
                        estimate(*all_cols)).agg(funcs.avg("score")).show()
flights_raw.filter("is_late = 0").sample(True, 0.1).withColumn('score',
                        estimate(*all_cols)).agg(funcs.avg("score")).show()

Py4JJavaError: An error occurred while calling o325.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 18.0 failed 1 times, most recent failure: Lost task 1.0 in stage 18.0 (TID 133, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/talfranji/Downloads/spark-2.4.4-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/Users/talfranji/Downloads/spark-2.4.4-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/talfranji/Downloads/spark-2.4.4-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 286, in dump_stream
    for series in iterator:
  File "<string>", line 1, in <lambda>
  File "/Users/talfranji/Downloads/spark-2.4.4-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 101, in <lambda>
    retur

# Score on full data set - take 3
Use column names

In [48]:
import re
import pyspark.sql.dataframe
import pyspark.sql.functions as funcs

def _spark_extract_column_name(col):
    # pyspark.sql.Column is a class used to build expression
    # interestly there is no convenient way to get the name of the column, if exists
    # We us str() to extract that name
    s = str(col)
    m = re.match(r"Column\<[^']'(.*)'\>", s)
    if not m:
        return "_BAD_"
    return m.group(1)

def spark_extract_column_names(*spark_cols):
    """for given Column objects - return list of names"""
    return [_spark_extract_column_name(c) for c in spark_cols]


In [50]:
#### Scoring using pandas_udf
#### Take 3
import pyspark.sql.functions as funcs

all_cols = [flights_raw[c] for c in flights_raw.columns]
column_names = spark_extract_column_names(*all_cols)
column_names_bc = sc.broadcast(column_names)
model_bc = sc.broadcast(model)

@funcs.pandas_udf('double')
def estimate(*cols):
    column_names = column_names_bc.value
    X = pd.concat(cols[:-1], axis=1)
    X.columns = column_names[:-1]  # copy names for the model
    Y = cols[-1]
    Y.name = column_names[-1]
    model = model_bc.value
    score = model.score(X, Y)
    return pd.Series(score, index=range(X.shape[0]))


flights_raw.filter("is_late = 1").sample(True, 0.01).withColumn('score', estimate(*all_cols)).agg(funcs.avg("score")).show()
flights_raw.filter("is_late = 0").sample(True, 0.01).withColumn('score', estimate(*all_cols)).agg(funcs.avg("score")).show()



+--------------------+
|          avg(score)|
+--------------------+
|0.002277143551129...|
+--------------------+

+------------------+
|        avg(score)|
+------------------+
|0.9994762712696419|
+------------------+



# Trick - put code from notebook into module
Save a paragraph into a file

In [42]:
def LastParagraphToPyModule(py_filename, spark_context=None, src_text=None):
    """Write the last evaluated notebook paragraph to a file and send to executors"""
    if src_text is None:
        src_text = In[-2]  # last paragraph
    with open(py_filename, "w+t") as out_py:
        out_py.write(src_text)
    if spark_context:
        spark_context.addPyFile(py_filename)


In [44]:
LastParagraphToPyModule("date_transformer.py", sc)

# Trick - Debug pandas_udf func using this
Helpper function to try a pandas_udf even without spark

In [66]:
def local_pandas_udf_on_pddf(pd_df, spark_pandas_udf, *spark_cols):
    """For quick syntax debugging: 
       Activate a @pandas_udf function locally (not in Spark executor)
       Over a local pd.DataFrame. If a spark-DataFrame is given 
       take some data using spark_df.limit(10000).toPandas()"""
    if spark_pandas_udf.evalType != funcs.PandasUDFType.SCALAR:
        raise ValueError("local_pandas_udf_on_pddf - should be a PandasUDFType.SCALAR")
    if isinstance(pd_df, pyspark.sql.dataframe.DataFrame):
        # not a pandas.DataFrame - get some data
        pd_df = pd_df.limit(10000).toPandas()
    cols = []
    for i, spark_c in enumerate(spark_cols):
        name = _spark_extract_column_name(spark_c)
        ser = pd_df[name]
        # "hide" the name - as this what happens in spark executor
        # some models use name and need to fail in this test
        ser.name = "_%d" % i
        cols.append(ser)
    # @pandas_udf wraps the original function - which resides in .func field  
    # the function accepts series as parameters
    res = spark_pandas_udf.func(*cols)
    assert(res.shape[0] == cols[0].shape[0])
    return res
